In [2]:
import numpy as np
import random as rd


The removers:

In [2]:

    
def random_removal(solution, problem, q):
    """This function randomly removes q nodes from the solution. It returns the solution without the removed nodes, as well as the list of removed nodes.

    Args:
        solution (list): The solution containing nodes as a list of tuples.
        problem (dict): The problem dictionary containing nodes.
        q (int): The number of nodes to remove.
        
    Returns:
        tuple: A tuple containing the updated solution without removed nodes and the list of removed nodes.
    """
    
    nodes = problem["Nodes"]
    sol = solution.copy()
    
    to_remove = rd.sample(nodes, q)
    
    for node in to_remove:
        sol.remove(node)
    
    return sol, to_remove


In [13]:
def largest_trip_removal(solution,problem,q):
    """This function picks q costly trips, and removes the node which is the "furthest" away. Will have to find a smart way to calculate the furthest node. 
    
    Initial thoughts is to find the 'center' of the trip, and pick the largest outlier except from the depot of cource. For instance when traveling from lagunen (d) to three nodes in bergen sentrum and one in
    bønes. The one in bønes is furthest away from the center even though it is close to the depot. Since there are multiple nodes in sentrum they pull heavy, and will pull the center towwards them

    PSEUDOCODE: 
    
    1. Loop q times
    2. Select the largest trip
    3. Look though the selected trip and find the node which increases the length the most. 
    4. Remove the node
    
      
    
    Args:
        solution (list): The generated solution
        problem (dict): Problem file
        q (int): amount of nodes to remove
        
    Returns:
        removed_sol (list): The solution without the removed nodes.
        removed_nodes (list): The removed nodes.
    """
    
    
    pp = problem["PreProseccing"]
    
    trips = split_solution(solution)
    
    remove_list = []
    
    for _ in range(q):
        
        trip_idx = 0
        length = 0
        
        
        
        # Find the largest trip
        for j in range(len(trips)-1):
            if len(trips[j]) > length:
                trip_idx =j
                length = len(trips[j])
        
        
        
        
        to_remove = None
        max_increase = 0
        
        trip = trips[trip_idx]
        
        padded = [(0,0)]+trip+[(0,0)]
        
        
        
        for i in range(1,len(padded)-1):
            p0 = padded[i-1]
            p1 = padded[i]
            p2 = padded[i+1]
            
            increase,_ = detour_length_pp(p0,p1,p2,pp)

            if increase > max_increase:
                to_remove = p1
                max_increase = increase
        
        if to_remove == None:
            continue
        else:
            
            remove_list.append(to_remove)
        
            trip.remove(to_remove)
            trips[trip_idx] = trip
        
        
        
    new_sol = combine_solution(trips)
    
    return new_sol, remove_list

In [5]:
def similarity_removal(solution, problem,q):
    """This function removes q nodes from the solution based on the similarity between the nodes. The nodes with the highest similarity are removed.
    It selects a node at random and removes the q-1 nodes that are most similar to it. 
    The similarity accounts for:
        -the location of the nodes
        -the number of crew
        -the distance from the depot
        -the distance from the nearest charging station
        

    Args:
        solution (_type_): _description_
        problem (_type_): _description_
        q (_type_): _description_
    """
    locations = problem["Locations"]
    nodes = problem["Nodes"]
    pp = problem["PreProseccing"]
    d_matrix,cs_matrix,dwc_matrix = pp
    jobs = problem["Jobs"]
    
    
    chosen = rd.choice(nodes)
    loc_diff = []
    crew_diff = []
    depot_dist = []
    cs_dist = []
    
    
    crew_0 = jobs[chosen[0]]
    depot_dist_0 = d_matrix[0,chosen[0]]
    cs_dist_0 = cs_matrix[chosen[0],chosen[0]][1]
    
    for i in range(len(nodes)):
        node = nodes[i]
        crew_1 = jobs[node[0]]
        depot_dist_1 = d_matrix[0,node[0]]
        cs_dist_1 = cs_matrix[node[0],node[0]][1]
        
        loc_diff.append(d_matrix[chosen[0],node[0]])
        crew_diff.append(abs(crew_0-crew_1))
        depot_dist.append(abs(depot_dist_0-depot_dist_1))
        cs_dist.append(abs(cs_dist_0-cs_dist_1))
    
    loc_diff = normalize(loc_diff)
    crew_diff = normalize(crew_diff)
    depot_dist = normalize(depot_dist)
    cs_dist = normalize(cs_dist)
    
    similarity = [sum(x) for x in zip(loc_diff,crew_diff,depot_dist,cs_dist)]
    
    remove_list = []
    sol = solution.copy()
    
    for i in range(q):
        m = min(similarity) #indeksen til den mest like
        
        
        idx = similarity.index(m)
        similarity[idx] = 1000
        
        to_remove = nodes[idx]
       
        remove_list.append(to_remove)
        
        sol.remove(to_remove)
        
    
        
    return sol, remove_list

The inserters:

In [3]:
def greedy_insert(solution, to_insert, problem):
    """
    This function selects the greediest place to insert q removed nodes.
    The function checks all trips and inserts the node into the trip that will increase the distance the least.
    The node will not be placed in a trip if it breaks the passenger capacity and range of the largest vessel.

    Args:
        solution (list): The list of trips to insert the nodes into.
        to_insert (list): The nodes removed from the solution by the removal function.
        problem (dict): The problem file.

    Returns:
        list: The updated solution with the inserted nodes.
        
        
        
    ############
    Further work:
    
    This should take into account the different costs of the vesseltypes. At the moment this insentivises the use of the larger vessels, and will probably not see the whole solution space.
    ALSO SHOULD NOT BREAK RULE 2 IN THE NODE ORDER RULES
    """
    
    
    
    vesseltypes = problem["Vessels"]
    vessel = vesseltypes[max(vesseltypes.keys())]
    _, max_range, _, _, max_cap,_ = vessel
    pp = problem["PreProseccing"]
    
    trips = split_solution(solution)
    
    #print(f'Inserting {to_insert} into solution {solution}')
    
    for node in to_insert:
        lowest_increase = float("inf")
        modified_trip = 0
        chosen_trip = 0
        
        #print(f'Node: {node}')
        
        
        
        
        for t in range(len(trips)-1):
            
            trip = [(0,0)]+ trips[t] + [(0,0)]
            
            trip_len,_ = tourlength_pp(trip,pp)
            old_vessel, f = allocate_single_vessel(trips[t],problem)
            
            if not f:
                print("ERROR: Vessel not found")
                print('Trip#:',t)
                print('Trip:',trips[t])
                print(f'to_insert: {to_insert}')
            
            if old_vessel == 0:
                old_cost = 0
            else:
                
                old_cost = trip_len * vesseltypes[old_vessel][2]
            
            loc, binary = node
            
            if binary == 0:
                reverse = (loc,1)
            else:
                reverse = (loc,0)
            
            lower = 0
            higher = len(trip)-1
            
            if reverse in trip:
                #print('Reverse in trip')
                if binary == 0:
                    higher = trip.index(reverse)
                else:
                    lower = trip.index(reverse)+1
            
            for i in range(lower,higher):
                p1 = trip[i]
                p2 = trip[i+1]
                #print(f'Old Trip: {trip}')
                
                new_trip,index = detour(p1,node,trip)
                #print(f'New trip: {new_trip}')
                
                
                detour_len = detour_length_pp(p1[0],p2[0],node[0],pp)
        
                new_len = trip_len + detour_len
                
                max_pass = single_max_pass(new_trip,problem)
                
                #print(f'Max pass: {max_pass}, range_needed: {range_needed}')
                #print(f'Passenger cap: {max_cap}, range: {max_range}')
                if max_pass <= max_cap:
                    #print('Accepted')
                    
                    new_vessel, _ = allocate_single_vessel(new_trip[1:-1],problem) #add max pass
                    new_cost = new_len * vesseltypes[new_vessel][2]
                    
                    increase = new_cost - old_cost
                    

                    if increase < lowest_increase:
                        lowest_increase,modified_trip,chosen_trip = new_len,new_trip,t # -1 because of the depot
                        
                        
        #print(f"Node {node} is placed in modified trip {modified_trip}") 
        #print(f'Trip: {trips[best_trip]}')
        
        if modified_trip == 0:
            trips[-1].append(node)
        else:
            trips[chosen_trip] = modified_trip[1:-1]
        
        
    
    new_sol = combine_solution(trips)
    #print("Compleated greedy insert. New solution: ",new_sol)
    
    return new_sol

Remove-Insert pairs:

In [5]:
def random_remove_random_insert(solution,problem,q):
    
    removed_sol, removed = random_removal(solution,problem,q)
    
    new_sol = random_insert(removed_sol,problem,removed)
    
    return new_sol

In [4]:
def random_remove_greedy_insert(solution,problem,q):
    
    removed_sol, removed = random_removal(solution,problem,q)
    
    new_sol = greedy_insert(removed_sol,removed,problem)
    
    return new_sol

In [9]:
def largest_trip_remove_greedy_insert(solution,problem,q):
    
    removed_sol, removed = largest_trip_removal(solution,problem,q)
    
    new_sol = greedy_insert(removed_sol,removed,problem)
    
    return new_sol

In [ ]:
def largest_trip_remove_random_insert(solution,problem,q):
    
    removed_sol, removed = largest_trip_removal(solution,problem,q)
    
    new_sol = random_insert(removed_sol,problem,removed)
    
    return new_sol

In [3]:
def similarity_removal_greedy_insert(solution,problem,q):
    
    removed_sol, removed = similarity_removal(solution,problem,q)
    
    new_sol = greedy_insert(removed_sol,removed,problem)
    
    return new_sol

Escape operator:

In [20]:
def escape(solution,problem,q):
    
    removed_sol, removed = random_removal(solution,problem,q)
    new_sol = send_to_dummy(removed_sol,removed)
        
        
    return new_sol

Helpers:

In [17]:
def split_solution(solution):
    sublists = []
    sublist = []
    
    for item in solution:
        if item == (0,0):
            sublists.append(sublist)
            sublist = []
        else:
            sublist.append(item)
    
    sublists.append(sublist)
    
    return sublists

def combine_solution(trips):
    combined_solution = []
    for trip in trips:
        combined_solution.extend(trip)
        combined_solution.append((0,0))
    
    if combined_solution and combined_solution[-1] == (0,0):
        combined_solution.pop()
    
    return combined_solution

In [4]:
def detour_length_pp(p1,p2,p3,pp):
    
    distances, _ , _ = pp
    
    old_length = distances[p1,p2]
    new_length = distances[p1,p2] + distances[p2,p3]
    
    return new_length - old_length

def tourlength_pp(tour,preproseccing):
    
    all_distances, _,_ = preproseccing
    tourlength = 0
    cum_length = [0]

    for i in range(len(tour)-1):
        p1 = tour[i][0]
        p2 = tour[i+1][0]
        tourlength += all_distances[p1,p2]
        cum_length.append(tourlength)
    
    return tourlength, cum_length
    
    

In [2]:
def send_to_dummy(solution,calls):
    
    trips = split_solution(solution)
    
    trips[-1].extend(calls)
    
    solution = combine_solution(trips)
    
    return solution

In [1]:
def normalize(array:list):
    max_val = max(array)
    min_val = min(array)
    
    norm = []
    
    for val in array:
        norm.append((val-min_val)/(max_val-min_val))
    
    return norm